In [13]:
from dataclasses import dataclass, field
from vi import Agent, Config, Simulation, HeadlessSimulation
import pygame as pg
import os
import numpy as np
from pygame.math import Vector2
import random
from multiprocessing import Pool
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import math

In [14]:
frame_dir = "frames"
os.makedirs(frame_dir, exist_ok=True)
frame_count = 0

def save_frame(screen):
    global frame_count
    pg.image.save(screen, os.path.join(frame_dir, f"frame_{frame_count:05d}.png"))
    frame_count += 1

In [4]:
class RecordingSimulation(Simulation):
    def __init__(self, config):
        super().__init__(config)
        self.frame_count = 0
        os.makedirs("frames", exist_ok=True)

    def after_update(self) -> None:
        # Draw everything to the screen
        self._all.draw(self._screen)

        if self.config.visualise_chunks:
            self.__visualise_chunks()

        # Save current frame as an image
        # pg.image.save(self._screen, f"frames/frame_{self.frame_count:05d}.png")

        # Update the screen with the new image
        pg.display.flip()

        self._clock.tick(self.config.fps_limit)

        current_fps = self._clock.get_fps()
        if current_fps > 0:
            self._metrics.fps._push(current_fps)

            if self.config.print_fps:
                print(f"FPS: {current_fps:.1f}")  # noqa: T201

        # Increment a frame counter (you may need to initialize it somewhere)
        self.frame_count += 1


In [5]:
@dataclass
class PPConfig(Config):
    #desired initial populations
    initial_prey: int = 100
    initial_predators: int = 50

    #parameters of the Lotka-Volterra equations
    alpha = 0.0055    # slower prey reproduction
    beta = 0.015     # more effective predators
    delta = 0.8     # predator reproduction chance
    gamma = 0.005    # predator death chance

config = PPConfig(image_rotation = True, movement_speed = 3.0, radius = 25, duration = 60*60*10, fps_limit = 0)
    

In [22]:
class Prey(Agent):
    def on_spawn(self):
        self.flee_strength = 1.5
        self.join_strength = 1.0
        self.state = "Wander"
        self.wander_direction_cooldown = 50
        self.gender = random.choice(["Male", "Female"])
        return super().on_spawn()
    
    def change_position(self):
        self.there_is_no_escape()

        flee_force = Vector2(0, 0)

        neighbors = list(self.in_proximity_accuracy())
        nearby_partners = [(agent, distance) for agent, distance in neighbors if isinstance(agent, Prey) and (agent.gender != self.gender)]
        if any(isinstance(agent, Predator) for agent, _ in neighbors):
            self.state = "Flee"
        elif nearby_partners != [] and random.random() < self.config.alpha:
            self.state = "Sexual Reproduction"
        else:
            self.state = "Wander"


        if self.state == "Flee":
            self.wander_direction_cooldown = 0
            predators_nerby = [(agent, dist) for agent, dist in neighbors if isinstance(agent, Predator)]
            for predator, dist in predators_nerby:
                if dist == 0:
                    dist = 0.001
                direction = self.pos - predator.pos
                if dist > 0:
                    direction = direction / dist
                force_magnitude = self.flee_strength / dist
                flee_force += direction * force_magnitude
            
            self.move = flee_force
            if self.move.length() > 0:
                self.move = self.move.normalize() * self.config.movement_speed


        elif self.state == "Wander":
            if self.wander_direction_cooldown == 0:
                self.wander_direction_cooldown = 50
                wander_angle = random.uniform(0, 2 * math.pi)
                wander_force = Vector2(1, 0).rotate_rad(wander_angle)
                self.move = wander_force
                if self.move.length() > 0:
                    self.move = self.move.normalize() * self.config.movement_speed
            else:
                self.wander_direction_cooldown -= 1
        
        elif self.state == "Sexual Reproduction":
            closest_partner = min(nearby_partners, key=lambda x: x[1])[0]
            closest_partner.state = "Wait"
            direction = closest_partner.pos - self.pos
            if direction.length() > 0:
                direction = direction.normalize()
            self.move = direction * self.config.movement_speed
            self.reproduce()
            
        elif self.state == "Wait":
            #wait for partner to arrive
            self.move = Vector2(0, 0)


        self.pos += self.move

In [7]:
class Predator(Agent):
    def on_spawn(self):
        self.random_move_strength = 1.0
        self.predator_speed_boost = 1.0
        self.state = "Wander"
        self.wander_direction_cooldown = 0
        return super().on_spawn()
    
    def change_position(self):
        self.there_is_no_escape()
        if random.random() < self.config.gamma:
            self.kill()
            return
        
        attract_force = Vector2(0, 0)

        neighbors = list(self.in_proximity_accuracy())

        if any(isinstance(agent, Prey) for agent, _ in neighbors):
            self.state = "Hunt"
        else:
            self.state = "Wander"

        if self.state == "Hunt":
            self.wander_direction_cooldown = 0
            prey_nearby = [(agent, dist) for agent, dist in neighbors if isinstance(agent, Prey)]
            the_closest_prey, closest_dist = min(prey_nearby, key=lambda x: x[1])
            if closest_dist == 0:
                    closest_dist = 0.001
            direction = the_closest_prey.pos - self.pos
                
            if closest_dist > 0:
                direction = direction / closest_dist

            force_magnitude = 1.0 / closest_dist
            attract_force += direction * force_magnitude
            if random.random() < self.config.beta and the_closest_prey.is_alive():
                the_closest_prey.kill()
                if random.random() < self.config.delta:
                    self.reproduce()
            
            self.move = attract_force

            if self.move.length() > 0:
                self.move = self.move.normalize() * self.config.movement_speed * self.predator_speed_boost
            
                

        elif self.state == "Wander":
            if self.wander_direction_cooldown == 0:
                self.wander_direction_cooldown = 50
                wander_angle = random.uniform(0, 2 * math.pi)
                wander_force = Vector2(1, 0).rotate_rad(wander_angle)
                self.move = wander_force
                if self.move.length() > 0:
                    self.move = self.move.normalize() * self.config.movement_speed * self.predator_speed_boost
            else:
                self.wander_direction_cooldown -= 1
        
        self.pos += self.move

In [56]:
(
    RecordingSimulation(
        config
    )
    .batch_spawn_agents(config.initial_predators, Predator, images=["images/Predator.png"])
    .batch_spawn_agents(config.initial_prey, Prey, images=["images/Prey.png"])
    .run()
)

ShapeError: unable to append to a DataFrame of width 6 with a DataFrame of width 0

In [8]:
class HeadlessSimulationRefined(HeadlessSimulation):
   def __init__(self, config):
      super().__init__(config)
      self.frame_count = 0
      self.prey_predator__per_frame = []

   def after_update(self) -> None:
      # Increment a frame counter (you may need to initialize it somewhere)
      
      # if self.frame_count % 10 == 0:
      num_prey = len([agent for agent in self._all if isinstance(agent, Prey)])
      num_predator = len([agent for agent in self._all if isinstance(agent, Predator)])
      self.prey_predator__per_frame.append((num_prey, num_predator))
      if self.frame_count % 100 == 0:
            print(num_prey, num_predator)
      self.frame_count += 1

   def run(self):
        """Run the simulation until it's ended by closing the window or when the `vi.config.Schema.duration` has elapsed."""
        self._running = True

        while self._running:
            self.tick()

        return self.prey_predator__per_frame

In [11]:
def run_simulation(config: Config) -> pl.DataFrame:
    return (
        HeadlessSimulationRefined(config)
        .batch_spawn_agents(config.initial_predators, Predator, images=["images/agent.png"])
        .batch_spawn_agents(config.initial_prey, Prey, images=["images/triangle.png"])
        .run()
    )

In [25]:
agents_per_frame = run_simulation(config)
df = pd.DataFrame(agents_per_frame, columns=['Prey', 'Predator'])
df['Time'] = range(len(df)) 

df_melted = df.melt(id_vars='Time', value_vars=['Prey', 'Predator'],
                    var_name='Species', value_name='Population')

sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
sns.lineplot(data=df_melted, x='Time', y='Population', hue='Species')
plt.title("Predator-Prey Populations Over Time")
plt.xlabel("Time Step")
plt.ylabel("Population")
plt.legend(title="Species")
plt.tight_layout()
plt.show()

100 50
77 46
62 45
42 44
35 29
25 21
23 16
20 15
19 8
17 7
18 6
16 6
14 6
14 5
13 2
13 2
13 1
13 0
13 0
13 0
13 0
13 0
13 0
13 0
15 0
15 0
16 0
16 0
16 0
18 0
18 0
18 0
18 0
18 0
18 0
18 0
20 0
20 0
20 0
20 0
20 0
20 0
20 0
20 0
20 0
20 0
20 0
20 0
21 0
22 0
22 0
23 0
25 0
25 0
25 0
27 0
28 0
28 0
28 0
28 0
28 0
28 0
30 0
31 0
31 0
31 0
31 0
31 0
32 0
32 0
35 0
37 0
39 0
41 0
41 0
47 0
50 0
51 0
54 0
58 0
64 0
71 0
75 0
77 0
82 0
86 0
88 0
94 0
105 0
115 0
127 0


KeyboardInterrupt: 